In [1]:
#############################################################################
# zlib License
#
# (C) 2023 Cristóvão Beirão da Cruz e Silva <cbeiraod@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

#############################################################################
# Modified for ETROC2 I2C testing in jupyter notebooks, Murtaza Safdari
#############################################################################

Imports

In [2]:
import logging
import i2c_gui
import i2c_gui.chips
from i2c_gui.usb_iss_helper import USB_ISS_Helper
from i2c_gui.fpga_eth_helper import FPGA_ETH_Helper

Set defaults

In [ ]:
# 'If set, the full log will be saved to a file (i.e. the log level is ignored)'
log_file = False
# 'Set the logging level. Default: WARNING',
#  ["CRITICAL","ERROR","WARNING","INFO","DEBUG","TRACE","DETAILED_TRACE","NOTSET"]
log_level_text = "WARNING"
# 'The port name the USB-ISS module is connected to. Default: COM3'
port = "COM3"
# I2C addresses for the pixel block and WS
chip_address = 0x60
ws_address = None

In [9]:
if log_file:
    logging.basicConfig(filename='logging.log', filemode='w', encoding='utf-8', level=logging.NOTSET)
    log_level = 0
else:
    log_level = 0
    if log_level_text == "CRITICAL":
        log_level=50
    elif log_level_text == "ERROR":
        log_level=40
    elif log_level_text == "WARNING":
        log_level=30
    elif log_level_text == "INFO":
        log_level=20
    elif log_level_text == "DEBUG":
        log_level=10
    elif log_level_text == "TRACE":
        log_level=8
    elif log_level_text == "DETAILED_TRACE":
        log_level=5
    elif log_level_text == "NOTSET":
        log_level=0
    logging.basicConfig(format='%(asctime)s - %(levelname)s:%(name)s:%(message)s')

i2c_gui.__no_connect__ = False  # Set to fake connecting to an ETROC2 device
i2c_gui.__no_connect_type__ = "echo"  # for actually testing readback
#i2c_gui.__no_connect_type__ = "check"  # default behaviour

Start logger and connect

In [10]:
logger = logging.getLogger("Script_Logger")

Script_Helper = i2c_gui.ScriptHelper(logger)

## USB ISS connection
conn = i2c_gui.Connection_Controller(Script_Helper)
conn.connection_type = "USB-ISS"
conn.handle: USB_ISS_Helper
conn.handle.port = port
conn.handle.clk = 100

## For FPGA connection (not yet fully implemented)
#conn.connection_type = "FPGA-Eth"
#conn.handle: FPGA_ETH_Helper
#conn.handle.hostname = "192.168.2.3"
#conn.handle.port = "1024"

conn.connect()

In [11]:
chip = i2c_gui.chips.ETROC2_Chip(parent=Script_Helper, i2c_controller=conn)
chip.config_i2c_address(chip_address)  # Not needed if you do not access ETROC registers (i.e. only access WS registers)
# chip.config_waveform_sampler_i2c_address(ws_address)  # Not needed if you do not access WS registers

logger.setLevel(log_level)

Show all peripheral registers

In [29]:
peripheralRegisterKeys = [i for i in range(32)]
for peripheralRegisterKey in peripheralRegisterKeys:
    handle_PeriCfgX = chip.get_display_var("ETROC2", "Peripheral Config", f"PeriCfg{peripheralRegisterKey}")
    chip.read_register("ETROC2", "Peripheral Config", f"PeriCfg{peripheralRegisterKey}")
    data_str_PeriCfgX = handle_PeriCfgX.get()
    data_int_PeriCfgX = int(data_str_PeriCfgX, base=16)
    data_bin_PeriCfgX = format(data_int_PeriCfgX, '08b')
    data_hex_PeriCfgX = hex(int(data_bin_PeriCfgX, base=2))
    print(f"PeriCfg{peripheralRegisterKey:2}", data_str_PeriCfgX, data_bin_PeriCfgX, f"{data_int_PeriCfgX:4}", data_hex_PeriCfgX)

PeriCfg 0 0x12 00010010   18 0x12
PeriCfg 1 0x98 10011000  152 0x98
PeriCfg 2 0x29 00101001   41 0x29
PeriCfg 3 0x18 00011000   24 0x18
PeriCfg 4 0x21 00100001   33 0x21
PeriCfg 5 0x00 00000000    0 0x0
PeriCfg 6 0x03 00000011    3 0x3
PeriCfg 7 0xa3 10100011  163 0xa3
PeriCfg 8 0xe3 11100011  227 0xe3
PeriCfg 9 0xe3 11100011  227 0xe3
PeriCfg10 0xd0 11010000  208 0xd0
PeriCfg11 0x10 00010000   16 0x10
PeriCfg12 0x00 00000000    0 0x0
PeriCfg13 0x80 10000000  128 0x80
PeriCfg14 0xf0 11110000  240 0xf0
PeriCfg15 0x60 01100000   96 0x60
PeriCfg16 0x90 10010000  144 0x90
PeriCfg17 0x98 10011000  152 0x98
PeriCfg18 0x60 01100000   96 0x60
PeriCfg19 0x42 01000010   66 0x42
PeriCfg20 0x40 01000000   64 0x40
PeriCfg21 0x2c 00101100   44 0x2c
PeriCfg22 0x00 00000000    0 0x0
PeriCfg23 0x00 00000000    0 0x0
PeriCfg24 0x00 00000000    0 0x0
PeriCfg25 0x00 00000000    0 0x0
PeriCfg26 0xaa 10101010  170 0xaa
PeriCfg27 0x8c 10001100  140 0x8c
PeriCfg28 0xc7 11000111  199 0xc7
PeriCfg29 0xac 101011

Single byte flip peripheral

In [54]:
peripheralRegisterKeys = [i for i in range(32)]
for peripheralRegisterKey in peripheralRegisterKeys:
    # Fetch the register
    handle_PeriCfgX = chip.get_display_var("ETROC2", "Peripheral Config", f"PeriCfg{peripheralRegisterKey}")
    chip.read_register("ETROC2", "Peripheral Config", f"PeriCfg{peripheralRegisterKey}")
    data_bin_PeriCfgX = format(int(handle_PeriCfgX.get(), base=16), '08b')
    # Make the flipped bits
    # data_bin_modified_PeriCfgX = list(data_bin_PeriCfgX)
    data_bin_modified_PeriCfgX = data_bin_PeriCfgX.replace('1', '2').replace('0', '1').replace('2', '0')
    # data_bin_modified_PeriCfgX = ''.join(data_bin_modified_PeriCfgX)
    data_hex_modified_PeriCfgX = hex(int(data_bin_modified_PeriCfgX, base=2))
    # Set the register with the value
    handle_PeriCfgX.set(data_hex_modified_PeriCfgX)
    chip.write_register("ETROC2", "Peripheral Config", f"PeriCfg{peripheralRegisterKey}")
    # Perform two reads to verify the persistence of the change
    chip.read_register("ETROC2", "Peripheral Config", f"PeriCfg{peripheralRegisterKey}")
    data_bin_new_1_PeriCfgX = format(int(handle_PeriCfgX.get(), base=16), '08b')
    chip.read_register("ETROC2", "Peripheral Config", f"PeriCfg{peripheralRegisterKey}")
    data_bin_new_2_PeriCfgX = format(int(handle_PeriCfgX.get(), base=16), '08b')
    # Undo the change to recover the original register value, and check for consistency
    handle_PeriCfgX.set(hex(int(data_bin_PeriCfgX, base=2)))
    chip.write_register("ETROC2", "Peripheral Config", f"PeriCfg{peripheralRegisterKey}")
    chip.read_register("ETROC2", "Peripheral Config", f"PeriCfg{peripheralRegisterKey}")
    data_bin_recover_PeriCfgX = format(int(handle_PeriCfgX.get(), base=16), '08b')
    # Handle what we learned from the tests
    print(f"PeriCfg{peripheralRegisterKey:2}", data_bin_PeriCfgX, "To", data_bin_new_1_PeriCfgX,  "To", data_bin_new_2_PeriCfgX, "To", data_bin_recover_PeriCfgX)
    if(data_bin_new_1_PeriCfgX!=data_bin_new_2_PeriCfgX or data_bin_new_2_PeriCfgX!=data_bin_modified_PeriCfgX or data_bin_recover_PeriCfgX!=data_bin_PeriCfgX): 
       print(f"PeriCfg{peripheralRegisterKey:2}", "FAILURE")


PeriCfg 0 00010010 To 11101101 To 11101101 To 00010010
PeriCfg 1 10011000 To 01100111 To 01100111 To 10011000
PeriCfg 2 00101001 To 11010110 To 11010110 To 00101001
PeriCfg 3 00011000 To 11100111 To 11100111 To 00011000
PeriCfg 4 00100001 To 11011110 To 11011110 To 00100001
PeriCfg 5 00000000 To 11111111 To 11111111 To 00000000
PeriCfg 6 00000011 To 11111100 To 11111100 To 00000011
PeriCfg 7 10100011 To 01011100 To 01011100 To 10100011
PeriCfg 8 11100011 To 00011100 To 00011100 To 11100011
PeriCfg 9 11100011 To 00011100 To 00011100 To 11100011
PeriCfg10 11010000 To 00101111 To 00101111 To 11010000
PeriCfg11 00010000 To 11101111 To 11101111 To 00010000
PeriCfg12 00000000 To 11111111 To 11111111 To 00000000
PeriCfg13 10000000 To 01111111 To 01111111 To 10000000
PeriCfg14 11110000 To 00001111 To 00001111 To 11110000
PeriCfg15 01100000 To 10011111 To 10011111 To 01100000
PeriCfg16 10010000 To 01101111 To 01101111 To 10010000
PeriCfg17 10011000 To 01100111 To 01100111 To 10011000
PeriCfg18 

Single byte flip pixel

In [56]:
pixelRegisterKeys = [i for i in range(32)]
row_indexer_handle,_,_ = chip.get_indexer("row")  # Returns 3 parameters: handle, min, max
column_indexer_handle,_,_ = chip.get_indexer("column")
for row in range(16):
    for col in range(16):
        print("Pixel", row, col)
        column_indexer_handle.set(col)
        row_indexer_handle.set(row)
        for pixelRegisterKey in pixelRegisterKeys:
            # Fetch the register
            handle_PixCfgX = chip.get_indexed_var("ETROC2", "Pixel Config", f"PixCfg{pixelRegisterKey}")
            chip.read_register("ETROC2", "Pixel Config", f"PixCfg{pixelRegisterKey}")
            data_bin_PixCfgX = format(int(handle_PixCfgX.get(), base=16), '08b')
            # Make the flipped byte
            data_bin_modified_PixCfgX = data_bin_PixCfgX.replace('1', '2').replace('0', '1').replace('2', '0')
            data_hex_modified_PixCfgX = hex(int(data_bin_modified_PixCfgX, base=2))
            # Set the register with the value
            handle_PixCfgX.set(data_hex_modified_PixCfgX)
            chip.write_register("ETROC2", "Pixel Config", f"PixCfg{pixelRegisterKey}")
            # Perform two reads to verify the persistence of the change
            chip.read_register("ETROC2", "Pixel Config", f"PixCfg{pixelRegisterKey}")
            data_bin_new_1_PixCfgX = format(int(handle_PixCfgX.get(), base=16), '08b')
            chip.read_register("ETROC2", "Pixel Config", f"PixCfg{pixelRegisterKey}")
            data_bin_new_2_PixCfgX = format(int(handle_PixCfgX.get(), base=16), '08b')
            # Undo the change to recover the original register value, and check for consistency
            handle_PixCfgX.set(hex(int(data_bin_PixCfgX, base=2)))
            chip.write_register("ETROC2", "Pixel Config", f"PixCfg{pixelRegisterKey}")
            chip.read_register("ETROC2", "Pixel Config", f"PixCfg{pixelRegisterKey}")
            data_bin_recover_PixCfgX = format(int(handle_PixCfgX.get(), base=16), '08b')
            # Handle what we learned from the tests
            if(data_bin_new_1_PixCfgX!=data_bin_new_2_PixCfgX or data_bin_new_2_PixCfgX!=data_bin_modified_PixCfgX or data_bin_recover_PixCfgX!=data_bin_PixCfgX): 
                print(row,col,f"PixCfg{pixelRegisterKey:2}","FAILURE", data_bin_PixCfgX, "To", data_bin_new_1_PixCfgX,  "To", data_bin_new_2_PixCfgX, "To", data_bin_recover_PixCfgX)

Pixel 0 0
Pixel 0 1
Pixel 0 2
Pixel 0 3
Pixel 0 4
Pixel 0 5
Pixel 0 6
Pixel 0 7
Pixel 0 8
Pixel 0 9
Pixel 0 10
Pixel 0 11
Pixel 0 12
Pixel 0 13
Pixel 0 14
Pixel 0 15
Pixel 1 0
Pixel 1 1
Pixel 1 2
Pixel 1 3
Pixel 1 4
Pixel 1 5
Pixel 1 6
Pixel 1 7
Pixel 1 8
Pixel 1 9
Pixel 1 10
Pixel 1 11
Pixel 1 12
Pixel 1 13
Pixel 1 14
Pixel 1 15
Pixel 2 0
Pixel 2 1
Pixel 2 2
Pixel 2 3
Pixel 2 4
Pixel 2 5
Pixel 2 6
Pixel 2 7
Pixel 2 8
Pixel 2 9
Pixel 2 10
Pixel 2 11
Pixel 2 12
Pixel 2 13
Pixel 2 14
Pixel 2 15
Pixel 3 0
Pixel 3 1
Pixel 3 2
Pixel 3 3
Pixel 3 4
Pixel 3 5
Pixel 3 6
Pixel 3 7
Pixel 3 8
Pixel 3 9
Pixel 3 10
Pixel 3 11
Pixel 3 12
Pixel 3 13
Pixel 3 14
Pixel 3 15
Pixel 4 0
Pixel 4 1
Pixel 4 2
Pixel 4 3
Pixel 4 4
Pixel 4 5
Pixel 4 6
Pixel 4 7
Pixel 4 8
Pixel 4 9
Pixel 4 10
Pixel 4 11
Pixel 4 12
Pixel 4 13
Pixel 4 14
Pixel 4 15
Pixel 5 0
Pixel 5 1
Pixel 5 2
Pixel 5 3
Pixel 5 4
Pixel 5 5
Pixel 5 6
Pixel 5 7
Pixel 5 8
Pixel 5 9
Pixel 5 10
Pixel 5 11
Pixel 5 12
Pixel 5 13
Pixel 5 14
Pixel 5 15
Pixe

In [ ]:
# ### Global status registers can only be read, not written
# handle_PeriSta0 = chip.get_display_var("ETROC2", "Peripheral Status", "PeriSta0")
# chip.read_register("ETROC2", "Peripheral Status", "PeriSta0")
# print(handle_PeriSta0.get())

# ### Read a pixel status register (remember that the indexers are used in the background to choose which pixel is being operated on)
# handle_PixSta1_2_4 = chip.get_indexed_var("ETROC2", "Pixel Status", "PixSta1")
# chip.read_register("ETROC2", "Pixel Status", "PixSta1")
# print(handle_PixSta1_2_4.get())

In [7]:
### You may also be interested in the following functions:
## Read and write the full chip, you can use handles in the middle to modify individual registers and then write everything at once
#chip.read_all()
#chip.write_all()
## Read and write a full address space:
#chip.read_all_address_space("ETROC2")
#chip.write_all_address_space("ETROC2")
## Read and write a full block within an address space (Status blocks can not be written to)
#chip.read_all_block("ETROC2", "Pixel Config:3:4") # Block name for a spacific pixel
#chip.read_all_block("ETROC2", "Pixel Config") # Block name for all pixels
#chip.write_all_block("ETROC2", "Pixel Config") # Block name for all pixels

In [57]:
conn.disconnect()